In [2]:
import sys
sys.path.append('..')

# Milestone 2

## Introduction

Automatic differentiation is a tool for calculating derivatives using machine accuracy. It has several advantages over traditional methods of derivative calculations such as symbolic and finite differentiation. Automatic differentiation is useful for calculating complex derivatives where errors are more likely with classical methods. For instance , with finite differentiation, h values that are too small will lead to accuracy errors though floating point roundoff error, while h values that are too large will start making vastly inaccurate approximations. 

Automatic differentiation is useful due to its practicality in real world applications that involve thousands of parameters in a complicated function, which would take a long runtime as well as strong possibility for error in calculating the derivatives individually. 

Our package allows users to calculate derivatives of complex functions, some with many parameters, allowing machine precision.

## Background

Essentially automatic differentiation works by  breaking down a complicated function and performing a sequence of elementary arithmetic such as addition, subtraction, multiplication, and division as well as elementary functions like exp, log, sin, etc. These operations are then repeated by the chain rule and the derivatives of these sequences are calculated. There are two ways that automatic differentiation can be implemented - forward mode and reverse mode. 


### 2.1 The Chain Rule

The chain rule makes up a fundamental component of auto differentiation. The basic idea is:   
For univariate function, $$ F(x) = f(g(x))$$
 $$F^{\prime} = (f(g))^{\prime} = f^{\prime}(g(x))g^{\prime}(x)$$
For multivariate function, $$F(x) = f(g(x),h(x))$$
$$ \frac{\partial F}{\partial x}=\frac{\partial f}{\partial g} \frac{\partial g}{\partial x}+\frac{\partial f}{\partial h} \frac{\partial h}{\partial x}$$
For more generalized cases, if F is a combination of more sub-functions,  $$F(x) = f(g_{1}(x), g_{2}(x), …, g_{m}(x))$$
$$\frac{\partial F}{\partial x}=\sum_{i=1}^{m}\frac{\partial F}{\partial g_{i}} \frac{\partial g_{i}}{\partial x}$$

### 2.2 Forward Mode

The forward mode automatic differentiation is accomplished by firstly splitting the function process into one-by-one steps, each including only one basic operation. Then from the first node, the value and derivative will be calculated based on the values and derivatives of forward nodes. AD exploits the fact that every computer program, no matter how complicated, executes a sequence of elementary arithmetic operations (addition, subtraction, multiplication, division, etc.) and elementary functions (exp, log, sin, cos, etc.). By applying the chain rule repeatedly to these operations, derivatives of arbitrary order can be computed automatically, accurately to working precision, and using at most a small constant factor more arithmetic operations than the original program.

An example of computational graph and table for forward mode AD is shown as follows:

\begin{align}
  f\left(x,y\right) =\sin\left(xy\right)
\end{align}
We will be evaluating the function at $f(1, 0)$

Evaluation trace:

| Trace   | Elementary Function      | Current Value           | Elementary Function Derivative       | $\nabla_{x}$ Value  | $\nabla_{y}$ Value  |
| :---: | :-----------------: | :-----------: | :----------------------------: | :-----------------:  | :-----------------: |
| $x_{1}$ | $x_{1}$                  | $1$        | $\dot{x}_{1}$                        | $1$ | $0$ |
| $x_{2}$ | $x_{2}$                  | $0$        | $\dot{x}_{2}$                        | $0$ | $1$ |
| $x_{3}$ | $x_{1}x_{2}$                  | $0$        | $\dot{x}_{2}$                        | $0$ | $1$ |

![comp-graph](computationalgraph.png)



### 2.3 Reverse Mode

The reverse mode automatic differentiation has a process similar to the forward mode auto differentiation, but has another reverse process. It does not apply the chain rule and only partial derivatives to a node are stored. First, for the forward process, the partial derivatives are stored for each node. For the reverse process, it starts with the differentiation to the last node, and then activations in the forward process are deployed in the differentiation differentiation step by step. 


### 2.4 Forward Mode v.s. Reverse Mode

Two main aspects can be considered when choosing between Forward and Reverse mode auto differentiation.
* Memory Storage & Time of Computation

The forward mode needs memory storage for values and derivatives for each node, while the reverse mode only needs to store the activations of partial differentiation to each node. The forward mode do the computation at the same time as the variable evaluation, while the reverse mode do the calculation in the backward process.
* Input & Output Dimensionality

If the input dimension is much larger than output dimension, then reverse mode is more attractive. If the output dimension is much larger than the input dimension, the forward mode is much computational cheaper.


## How to use VorDiff

Our Automatic Differentiation package is called VorDiff. The two main objects you will interact with are `AutoDiff` and `Operator`. In short, the user will first instantiate a scalar variable as an `AutoDiff` object, and then feed those variables to operators specified in the `Operator` object. The `Operator` object allows users to build their own functions for auto-differentiation. Simple operations (e.g. addition, multiplication, power) may be used normally. More complex functions (e.g. log, sin, cos) must use the operations defined in the `Operator` class. Lastly, the user may retrieve the values and first derivatives from the objects defined above by using the `get()` method.

A short example is provided below:

In [15]:
from VorDiff.autodiff import AutoDiff as ad
from VorDiff.operator import Operator as op

# Define variables
x = ad.scalar(3.14159)
y = ad.scalar(0)

# Build functions
fx = op.sin(x) + 3
fy = op.exp(y) + op.log(y+1)

# Get values and derivates
print(fx.get())
print(fy.get())

(3.0000026535897932, -0.9999999999964793)
(1.0, 2.0)


## Software Organization

### Directory Structure
The package's directory will be structured as follows:
```
VorDiff
	__init__.py
    nodes
	    node.py
	    scalar.py
	    vector.py
	tests
	    test_nodes
		    test_node.py
		    test_scaler.py
		    test_vector.py
		test_autodiff.py
        test_operator.py
	docs
		...
	demo
	    ...
	autodiff.py
    operator.py
    README.md
    ...
```
### Modules
-   VorDiff: The VorDiff module contains the operator class to be directly used by users to evaluate functions and calculate their derivatives, a class node and subclasses scalar and vector to be used in autodiff class, and an autodiff class to perform automatic differentiation. This is the core of the package.
    
-   Test_Vordiff: The Test_Vordiff module contains the test suite for this project. TravisCI and CodeCov are used to test our operator classes, node classes, and auto-differentiator.
    
-   Demo: The Demo module contains python files demonstrating how to perform automatic differentiation with the implemented functions.
    
### Testing
In this project we will use TravisCI to perform continuous integration testing and CodeCov to check the code coverage of our test suite. The status us TravisCI and CodeCov can be found in README.md, in the top level of our package. Since the test suite is included in the project distribution, users can also install the project package and use pytest and pytest-cov to check the test results locally.

### Distribution:
Our open-source VorDiff package will be uploaded to PyPI by using twine because it uses a verified connection for secure authentication to PyPI over HTTPS. Users will be able to install our project package by using the convential `pip install VorDiff`.



## Implementation

### Scalar
The `Scalar` class represents a single scalar node in the computational graph of a function. It implements the interface for user defined scalar variables. The object contains two hidden attributes, `._val` and `._der`, which can be retrieved with the `get()` method.

In [16]:
import numpy as np

# Documentation Hidden
class Scalar():

    def __init__(self, value, *kwargs):
        self._val = value
        if len(kwargs) == 0:
            self._der = 1
        else:
            self._der = kwargs[0]
    
    def get(self):
        return self._val, self._der

    def __add__(self, other):
        try:
            return Scalar(self._val+other._val, self._der+other._der)
        except AttributeError:
            return self.__radd__(other)

    def __radd__(self, other):
        return Scalar(self._val+other, self._der)
    
    def __mul__(self, other):
        try:
            return Scalar(self._val*other._val, self._der*other._val+self._val*other._der)
        except AttributeError:
            return self.__rmul__(other)
        
    def __rmul__(self, other):
        return Scalar(self._val*other, self._der*other)
    
    def __sub__(self, other):
        return self + (-other)
    
    def __rsub__(self, other):
        return -self + other
    
    def __truediv__(self, other):
        try:
            return Scalar(self._val/other._val, (self._der*other._val-self._val*other._der)/(other._val**2))
        except AttributeError:
            return Scalar(self._val/other, self._der/other)
    
    def __rtruediv__(self, other):
        return Scalar(other/self._val, other*(-self._der)/(self._val)**2)

    def __pow__(self, other):
        try:
            return Scalar(self._val**other._val, (other._val*self._der/self._val+np.log(self._val)*other._der)*(self._val**other._val))
        except AttributeError:
            return Scalar(self._val**other, other*(self._val**(other-1))*self._der)
            
    def __rpow__(self, other):
        return Scalar(other**self._val, (other**self._val)*np.log(other)*self._der)
    
    def __neg__(self):
        return Scalar((-1)*self._val, (-1)*self._der)

### Operator
The operator class contains all mathematical operations that users can call to build their functions. Each function returns a `Scalar` object or a numeric constant, depending on the input type. Each function raises an erro if its input falls outside its domain. All functions in the class are static.

In this implementation, we include the following elementary functions. Derivatives are calculated with the chain rule.

In [17]:
import numpy as np
from VorDiff.nodes.scalar import Scalar

# Documentation Hidden
class Operator():
    
    @staticmethod
    def sin(x):
        try: # If scalar variable
            return Scalar(np.sin(x._val), x._der*np.cos(x._val))
            
        except AttributeError: # If contant
            return np.sin(x)
        
    @staticmethod
    def cos(x):
        try: # If scalar variable
            return Scalar(np.cos(x._val), -np.sin(x._val)*x._der)
            
        except AttributeError: # If contant
            return np.cos(x)
        
    @staticmethod
    def tan(x):
        try: # If scalar variable
            return Scalar(np.tan(x._val), x._der/np.cos(x._val)**2)
            
        except AttributeError: # If contant
            return np.tan(x)
        
    @staticmethod
    def arcsin(x):
        try: # If scalar variable
            if x._val<-1 or x._val>1:
                raise ValueError('out of domain')
            else:
                return Scalar(np.arcsin(x._val), 1/(x._der*(1-x._val**2)**.5))
            
        except AttributeError: # If contant
            if x<-1 or x>1:
                raise ValueError('out of domain')
            else:
                return np.arcsin(x)
        
    @staticmethod
    def arccos(x):
        try: # If scalar variable
            if x._val<-1 or x._val>1:
                raise ValueError('out of domain')
            else:
                return Scalar(np.arccos(x._val), -x._der/(1-x._val**2)**.5)
            
        except AttributeError: # If contant
            if x<-1 or x>1:
                raise ValueError('out of domain')
            else:
                return np.arccos(x)
        
    @staticmethod
    def arctan(x):
        try: # If scalar variable
            return Scalar(np.arctan(x._val), x._der/(1+x._val**2))
            
        except: # If contant
            return np.arctan(x)
        
    @staticmethod
    def log(x):
        try: # If scalar variable
            return Scalar(np.log(x._val), x._der/x._val)
            
        except AttributeError: # If contant
            return np.log(x)
        
    @staticmethod
    def exp(x):
        try: # If scalar variable
            return Scalar(np.exp(x._val), x._der*np.exp(x._val))
            
        except AttributeError: # If contant
            return np.exp(x)
        
    @staticmethod
    def sinh(x):
        try: # if scalar variable
            return Scalar(np.sinh(x._val), x._der*(np.cosh(x._val)))
   
        except AttributeError: #if constant
            return np.sinh(x)  
      
    @staticmethod
    def cosh(x):
        try: # if scalar variable
            return Scalar(np.cosh(x._val), x._der*(np.sinh(x._val)))
   
        except AttributeError: #if constant
            return np.cosh(x)

    @staticmethod
    def tanh(x):
        try: # if scalar variable
            return Scalar(np.tanh(x._val), x._der*(1-np.tanh(x._val)**2))
   
        except AttributeError: #if constant
            return np.tanh(x)

    @staticmethod
    def arcsinh(x):
        try: # if scalar variable
            return Scalar(np.arcsinh(x._val), x._der*(-np.arcsinh(x._val)*np.arctanh(x._val)))
   
        except AttributeError: #if constant
            return np.arcsinh(x)
        
    @staticmethod
    def arccosh(x):
        try: # if scalar variable
            return Scalar(np.arccosh(x._val), x._der*(-np.arccosh(x._val)*np.tanh(x._val)))
   
        except AttributeError: #if constant
            return np.arccosh(x)
        
    @staticmethod
    def arctanh(x):
        try: # if scalar variable
            return Scalar(np.arctanh(x._val), x._der*(1-np.arctanh(x._val)**2))
   
        except AttributeError: #if constant
            return np.arctanh(x)

### AutoDiff
The `AutoDiff` class will allow the user to easily create variables and build auto-differentiable functions, without having to interface with the `Node` class. It will make use of the auto-differentiator much more intuitive for the user. It will become much more important when we implement `Vector` nodes, in addition to just `Scalar` nodes.

In [18]:
from VorDiff.nodes.scalar import Scalar

# Documentation Hidden
class AutoDiff():

    @staticmethod
    def scalar(val):
        
        return Scalar(val, 1)

## Installation

Our package can be installed from our GitHub Repository at: https://github.com/VoraciousFour/cs207-FinalProject.

After the package is installed, it needs to be imported into their workspace. Doing so, will automatically download any dependencies that are required by our package such as math or numpy. Then, the user can create and activate a virtual envitronment to use the package in.

The user can set up and use our package using their terminal as follows.

1. Clone the VorDiff package from our Github Repository into your directory
        git clone https://github.com/VoraciousFour/cs207-FinalProject.git
2. Create and activate a virtual environment
        '''Installing virtualenv'''
        sudo easy_install virtualenv
        '''Creating the Virtual Environment'''
        virtualenv env
        '''Activating the Virtual Environment'''
        source env/bin/activate
3. Install required dependencies
        pip install -r requirements.txt5
4. Importing VorDiff package for use
        import VorDiff

## Future Features

1. Method to install VorDiff using Python Package Index (PyPl) 


2. Jacobian will be changed to support multiple functions of multiple inputs instead of just scalar function of single input

 Adding a method to install VorDiff with PyPl will be simple as we will       navigate the pypi website and publish VorDiff there.

 Changing the Jacobian to support multiple functions will require us to       change some of our code in our implementations to work with multiple         functions and inputs.